In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import tensorflow as tf
from keras.callbacks import Callback

import copy



from tensorflow.keras.regularizers import l1, l2

from tqdm import tqdm

import numpy as np
import cv2
# import mediapipe as mp
from IPython.display import clear_output
from sklearn.model_selection import train_test_split

# from tensorflow_model_optimization.quantization.keras import quantize_model
from collections import Counter
import random as rand
import os

In [7]:
import numpy as np

def txt_pre_process(txt_file,label,simplify=False,simplify_level=14 ):
    label_array = []
    temp_feature_data = []
    temp_sequence_data = []
    batch_data = []

    with open(str(txt_file), 'r') as file:

        for line in file:
            values = line.strip().split('|')

            temp_feature_data = []

            for value in values:
                float_value = str(value)

                #FIRST PART OF THE SEQUENCE
                if float_value == 'START':
                    temp_sequence_data=[]

                elif float_value == 'END':
                    batch_data.append(temp_sequence_data)
                    label_array.append(label)


                elif float_value != '' and float_value != 'START':
                    if simplify:
                        float_value = round(float(value),simplify_level)
                    else:
                        float_value = float(value)
                    temp_feature_data.append(float_value)

            if temp_feature_data!=[]:
                temp_sequence_data.append(temp_feature_data)

    label_array = np.array(label_array)
    return [batch_data,label_array]

#--------------------------------------------------------------------------- paddingV1 --------------------------------------------------------------------------------
# padding can be improved probably...by using sequence
# minor issue:
# > is whether sequences had exceeded the intended number of sequences but is still right (it was performed right but slower(by an acceptable margin)) - not resolved
#    = temporary fix was just to truncate everything if it had exceeded the intended number of sequence for the sake of running it for now
#    = a reliable solution in theory could be that to randomly truncate in between the first and end sequence, in this way relevant data can be captured
def padding(pre_processed_input,optional_maxLength=0):
    padded_sequences = []
    if optional_maxLength != 0:
        max_length = optional_maxLength
    else:
        max_length = max(len(sequence) for sequence in pre_processed_input)

    for sequence in pre_processed_input:
        padding_length = max_length - len(sequence)
        if padding_length >= 0:
            padded_sequence = np.pad(sequence, ((0, padding_length), (0, 0)), mode='constant')

        else:
            padded_sequence = sequence[:max_length]
        padded_sequences.append(padded_sequence)
    padded_sequences = np.array(padded_sequences)

    return padded_sequences

#--------------------------------------------------------------------------- paddingV1 --------------------------------------------------------------------------------

# this is to merge correct executions and wrong executions and randomize their input and label
# positions of input and its corresponding label are the same
# introducing noise/wrong input makes the model more robust
def concatenate_randomize_batches(base_input,base_label,concat_input,concat_label):
    combined_inputs = np.concatenate((base_input,concat_input), axis = 0)
    combined_label = np.concatenate((base_label,concat_label), axis = 0)
    indices = np.random.permutation(len(combined_inputs))
    randomized_inputs = combined_inputs[indices]
    randomized_label = combined_label[indices]
    return [randomized_inputs,randomized_label]




def tally_sequence(sequence_array):
    tally_number = []
    tally_ctr = []

    for x in sequence_array:
        temp = len(x)
        if temp not in tally_number:
            tally_number.append(temp)
            tally_ctr.append(1)
        else:
            for y in range(len(tally_number)) :
                if temp == tally_number[y]:
                    tally_ctr[y] = tally_ctr[y] + 1

    tally_max = 0
    tally_number_arranged = []
    tally_ctr_arranged = []

    for x in range(len(tally_number)):
        # print(len(tally_ctr))
        tally_max = max(tally_ctr)
        for y in range(len(tally_number)):
            if tally_ctr[y] == tally_max:
                tally_number_arranged.append(tally_number[y])
                tally_ctr_arranged.append(tally_ctr[y])
                tally_ctr.pop(y)
                tally_number.pop(y)
                break

    total_ctr = 0
    for x in tally_ctr:
        total_ctr = total_ctr + x


    for x in range(len(tally_number_arranged)):
        print(tally_number_arranged[x],'-->',tally_ctr_arranged[x])


# outlier detection and removal (currently being used)
def common_length_sequence(sequences_array,threshold = 2):
    temp = []

    data = [len(seq) for seq in sequences_array]
    data_frequency = Counter(data)
    most_common_data = data_frequency.most_common()
    outlier_frequencies = [value for value, freq in data_frequency.items() if freq < threshold]
    most_common_values = [value for value, freq in most_common_data if freq >= threshold]

    print("Most Common Data Points:", most_common_values)
    print("Outlier Frequencies:", outlier_frequencies)

    for x in sequences_array:
        if len(x) in most_common_values:
            temp.append(x)
    print('-------------------applied frequency outlier detection-------------------')
    print("original num -> ", len(sequences_array))
    print("current num -> ", len(temp))
    print("removed num -> ", len(sequences_array) - len(temp))
    return temp

# outlier detection and removal (currently being used)
def apply_z_score(sequences_array,z_score_threshold = 1):
    data_points = []
    included_datapoints = []
    updated_sequences =[]

    for x in sequences_array:
        temp = len(x)
        if temp not in data_points:
            data_points.append(temp)

    data = np.array(data_points)
    mean_value = np.mean(data)
    standard_deviation = np.std(data)
    z_scores = (data - mean_value) / standard_deviation
    for x in range(len(z_scores)):
        if np.abs(z_scores[x]) <= z_score_threshold:
            included_datapoints.append(data[x])


    for x in sequences_array:
        if len(x) in included_datapoints:
            updated_sequences.append(x)
    print('-------------------applied z-score outlier detection-------------------')
    print("datapoints included -> ", included_datapoints)
    print("original num -> ", len(sequences_array))
    print("current num -> ", len(updated_sequences))
    print("removed num -> ", len(sequences_array) - len(updated_sequences))

    return updated_sequences




def paddingV2(sequences_array_input,optional_maxlength = 0):
    sequences_array = copy.deepcopy(sequences_array_input)


    output = []
    max_length = 0
    if optional_maxlength == 0:
        max_length = max(len(sequence) for sequence in sequences_array)
        expanded_max_length = int(max_length+ ((max_length) * .10))
    else:
        expanded_max_length = optional_maxlength

    # sequence = np.array(sequences_array)

    print(expanded_max_length)


    padding_length_before = 0
    padding_length_after = 0

    for seq in sequences_array:
        # print(seq)
        for x in range(expanded_max_length-len(seq)+1):
            padding_length_before = x
            padding_length_after = expanded_max_length - len(seq) - x
            padded_sequence = np.pad(seq, ((padding_length_before, padding_length_after),(0,0)), mode='constant')
            output.append(padded_sequence)

            # print(padded_sequence)
    print('------------------------applied paddingV2------------------------')
    print('max_length -> ', max_length)
    print('expanded_max_length -> ', expanded_max_length)
    print('original num set of sequences -> ', len(sequences_array))
    print('final num set of sequences -> ', len(output))

    output = np.array(output)
    return output




def convert_tf_to_tflite(tf_model,input_shape,test_dataset,name,id_number,validation_loss,validation_accuracy):
  model = tf.keras.models.load_model(tf_model)

  run_model = tf.function(lambda x: model(x))
  # This is important, let's fix the input size.
  BATCH_SIZE = input_shape[0]
  STEPS = input_shape[1]
  INPUT_SIZE = input_shape[2]
  concrete_func = run_model.get_concrete_function(
      tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model.inputs[0].dtype))

  # model directory.
  MODEL_DIR = "keras_lstm"
  model.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

  converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
  tflite_model = converter.convert()


  # Run the model with TensorFlow to get expected results.
  TEST_CASES = 10

  # Run the model with TensorFlow Lite
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  for i in range(TEST_CASES):
    expected = model.predict(test_dataset[i:i+1])
    interpreter.set_tensor(input_details[0]["index"], test_dataset[i:i+1, :, :])
    interpreter.invoke()
    result = interpreter.get_tensor(output_details[0]["index"])

    # Assert if the result of TFLite model is consistent with the TF model.
    np.testing.assert_almost_equal(expected, result, decimal=5)
    print("Done. The result of TensorFlow matches the result of TensorFlow Lite.")

    interpreter.reset_all_variables()



  temp = 'converted_model_'

  temp3 = temp + str(name) + id_number + "(loss_"+ str(round(validation_loss,3)) +")" + "(acc_"+  str(round(validation_accuracy,3 )) + ")" + '.tflite'
  # Save the TFLite model to a file
  with open(temp3, "wb") as f:
    f.write(tflite_model)
  # with open("converted_model.tflite", "wb") as f:
  #     f.write(tflite_model)


# this data augmentation replaces padded index with random inputs
def populate_0_input(correct_data_input,noise_data_input):
    correct_data = copy.deepcopy(correct_data_input)
    noise_data = copy.deepcopy(noise_data_input)

    print(len(correct_data))
    index = 10
    temp = []
    temp_compilation = []
    ctr = 0
    rand_modifier =0

    for set_sequence in tqdm(correct_data, desc="populate_0_input", leave=True):
        rand_modifier = rand.randint(0,len(noise_data))

        for x in range(len(set_sequence)):
            ctr = ctr + 1
            if set_sequence[x][0] == 0:
                temp.append(noise_data[rand_modifier-1][rand.randint(0,len(noise_data[rand_modifier-1])-1)])

            else:
                temp.append(set_sequence[x])

        temp_compilation.append(temp)
        temp =[]


    return temp_compilation



class CustomEarlyStopping(Callback):
  def __init__(self, accuracy_threshold=0.95, loss_threshold=0.10):
      super(CustomEarlyStopping, self).__init__()
      self.accuracy_threshold = accuracy_threshold
      self.loss_threshold = loss_threshold

  def on_epoch_end(self, epoch, logs=None):
      if logs is None:
          logs = {}

      if logs.get('val_accuracy') is None or logs.get('val_loss') is None:
          return

      if logs.get('val_accuracy') >= self.accuracy_threshold and logs.get('val_loss') <= self.loss_threshold:
          self.model.stop_training = True
          print(f"\nTraining stopped as validation accuracy reached {logs.get('val_accuracy'):.4f} "
                f"and validation loss reached {logs.get('val_loss'):.4f}")


# this data augmentation augments data to simulate if the sequence were follow,whether there is unecessary movements in between.
# this would also reinforce the model to detect certain action that were classified as correct instead of incorrect
# this augmentation is augmenting each sequence of a list of sequence
# 1 set of sequence(example-> 1 push up):
# [seq1,seq2,seq3,seq4]
# [seq1,seq2,NOISE_SEQ34,seq4]
def data_aug_sensitivity(sequence_array_list_input,noise_sequence_list_input,num_data_aug = 3,num_aug_in_1_seq = 3,noise_seq_len = 2):
  sequence_array_list = copy.deepcopy(sequence_array_list_input)
  noise_sequence_list = copy.deepcopy(noise_sequence_list_input)

  compile = []
  temp_seq = []
  temp_storage = []
  temp_rand = []
  num = 0
  ctr1111 = 0
  temp_rand2 = 0
  temp_rand3 = 0

# per sequences
  for sequence in tqdm(sequence_array_list, desc="data_aug_seq_sensitivity", leave=True):
    # loops for the number of data augmentation per sequence
    for ctr in range(num_data_aug):
      # loops for the amount of number of augmentation in the sequence(loops to get random index)
      while len(temp_rand)!=num_aug_in_1_seq:
        num = rand.randint(0,len(sequence)-1)
        if num in temp_rand:
          continue
        else:
          temp_rand.append(num)

      #actual augmentation of the sequence
      temp_seq = sequence.copy()
      # store in a temp variable and to be edited

      # number of augmentation to be done in a sequence
      for ctr1 in range(len(temp_rand)):

        # number of sequence to be expanded(index + number of noise_seq_len)
        for ctr2 in range(noise_seq_len):
          temp_rand2 = rand.randint(0,len(noise_sequence_list)-1)
          temp_rand3 = rand.randint(0,len(noise_sequence_list[0])-1)

          if (temp_rand[ctr1] + ctr2) < len(temp_seq):
            temp_seq[temp_rand[ctr1] + ctr2] = noise_sequence_list[temp_rand2][temp_rand3]


          else:
            continue


      compile.append(temp_seq)
      temp_seq = []
      temp_rand = []


  return compile


def data_aug_seq_sensitivity(sequence_array_list_input,num_to_aug=2,num_coor_edit=3,num_sequence_edit=2):
  sequence_array_list = copy.deepcopy(sequence_array_list_input)

  compile = []
  temp = []
  rand_coor = []


  for ctr in tqdm(range(num_to_aug), desc="data_aug_coor_sensitivity", leave=True):
    for sequence in sequence_array_list:
      for ctr3 in range(num_sequence_edit):
        what_sequence = rand.randint(0,len(sequence)-1)
        for ctr2 in range(num_coor_edit):
          what_coor = rand.randint(0,len(sequence[0])-1)
          rand_coor = rand.randint(0,9999999999)
          rand_coor = rand_coor / (10 ** len(str(rand_coor)))
          sequence[what_sequence][what_coor]=rand_coor
      compile.append(sequence)
  return compile


def data_aug_coor_sensitivity(sequence_array_list_input,num_coor_edit=45,num_sequence_edit=8):
  sequence_array_list = copy.deepcopy(sequence_array_list_input)

  compile = []
  temp = []
  rand_coor = []
  temp_seq = []



  # for ctr in tqdm(range(num_to_aug), desc="data_aug_coor_sensitivity", leave=True):
  for sequence in sequence_array_list:
    temp_seq = sequence.copy()
    for ctr3 in range(num_sequence_edit):
      what_sequence = rand.randint(0,len(sequence)-1)
      num_coor_edit = rand.randint(int(num_coor_edit*.65),num_coor_edit)
      for ctr2 in range(num_coor_edit):
        what_coor = rand.randint(0,len(sequence[0])-1)
        # rand_coor = rand.randint(0,9999999999)
        rand_coor = rand.randint(0,999)
        rand_coor = rand_coor / (10 ** len(str(rand_coor)))
        temp_seq[what_sequence][what_coor]=rand_coor
    compile.append(sequence)
  return compile




In [4]:
# this is as a whole exercise
def streamlined_process(correct_execution,noise_data):
  id_num = str(rand.randint(1000,9999))
  base_data = txt_pre_process(correct_execution,1,False,4)
  base_data_noise = txt_pre_process(noise_data,0,False,4)

  loop = 0

  temp_correct_start=[]
  temp_correct_wrong=[]

  temp_wrong_store_train = []

  data = base_data[0]
  data_noise = base_data_noise[0]

  best_val_loss = float('inf')
  best_val_accuracy = 0.0
  best_model = None



  print('')
  print('----------------------------correct data augmentation ----------------------------')
  aug = common_length_sequence(data)
  aug2 = apply_z_score(aug,1)
  aug3 = paddingV2(aug2)
  aug4 = populate_0_input(aug3,data_noise)
  aug4 = np.array(aug4)
  aug4 = aug4.reshape(-1,len(aug3[0]),len(aug3[0][0]))
  combined_inputs = np.concatenate((aug4,aug3), axis = 0)
  combined_inputs = aug4
  print('concat -> ', len(combined_inputs))


  print('')
  print('----------------------------data noise data augmentation ----------------------------')
  aug_noise_data1 = paddingV2(data_noise,len(aug3[0]))
  aug_noise_data2 = populate_0_input(aug_noise_data1,data_noise)

# testing...
  # data_aug_coor_sensitivity()

# original
  # aug_noise_data3 = aug_noise_data2[0:len(combined_inputs)]

# testing(temporary)
  aug_noise_data3 = aug_noise_data2[:]

  aug_noise_data4 = np.array(aug_noise_data3)

  aug_noise_data5 = aug_noise_data4.reshape(-1,len(aug_noise_data4[0]),len(aug_noise_data4[0][0]))

# testing(temporary)
  aug_noise_data7 = data_aug_coor_sensitivity(aug_noise_data5,num_sequence_edit=int(len(aug_noise_data5[0])*.70))

  # aug_noise_data6 = data_aug_sensitivity(aug4,aug_noise_data5,1,2,2)
  aug_noise_data6 = data_aug_sensitivity(aug4,aug_noise_data5,1,2,int(len(combined_inputs)*.35))

# original
  # aug_noise_data5 = np.concatenate((aug_noise_data6,aug_noise_data5), axis = 0)

# testing(temporary)
  aug_noise_data5 = np.concatenate((aug_noise_data6,aug_noise_data5,aug_noise_data7), axis = 0)


  print('aug_noise_data5--->',aug_noise_data5.shape)


  loop = int(len(aug_noise_data5)/len(combined_inputs))

  model = Sequential()
  model.add(LSTM(len(aug4[0]), return_sequences=True, activation='relu',  input_shape=(len(aug4[0]), len(aug4[0][0]))))
  model.add(LSTM(len(aug4[0]) + int(len(aug4[0]) - int(len(aug4[0]) * .4)), return_sequences=True,  activation='relu'))
  model.add(Bidirectional(LSTM(len(aug4[0]) - int(len(aug4[0]) - int(len(aug4[0]) * .4)), return_sequences=True, dropout=0.3, recurrent_dropout=0.3, activation='relu')))
  model.add(LSTM(len(aug4[0]) - int(len(aug4[0]) - int(len(aug4[0]) * .4)), return_sequences=False, activation='relu'))
  # model.add(BatchNormalization())
  model.add(Dense(len(aug4[0]) - int(len(aug4[0]) - int(len(aug4[0]) * .3)), activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  custom_early_stopping = CustomEarlyStopping(accuracy_threshold=0.97, loss_threshold=0.05)
  model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy'])




  for x in range(loop):
    temp_wrong_store_train = aug_noise_data5[x*len(combined_inputs):len(combined_inputs)*(x+1)]

    aug3_label = np.ones(len(combined_inputs))
    aug_noise_label = np.zeros(len(temp_wrong_store_train))

    rand_batches=concatenate_randomize_batches(combined_inputs,aug3_label,temp_wrong_store_train,aug_noise_label)

    X_train, X_test, y_train, y_test = train_test_split(rand_batches[0], rand_batches[1], test_size=0.3, random_state=42)

    # history=model.fit(X_train, y_train, epochs=200, batch_size =256  , validation_data=(X_test, y_test), callbacks=[custom_early_stopping])
    history=model.fit(X_train, y_train, epochs=200, batch_size =256  , validation_data=(X_test, y_test), callbacks=[custom_early_stopping])


    val_loss = min(history.history['val_loss'])
    val_accuracy = max(history.history['val_accuracy'])
    temp_wrong_store_train=[]


    if val_loss < best_val_loss and val_accuracy > best_val_accuracy:
      best_model = model.get_weights()
      best_val_loss = val_loss
      best_val_accuracy = val_accuracy

    # testing(temporary)
    model.set_weights(best_model)

  X_train = X_train.astype(np.float32)
  X_test = X_test.astype(np.float32)
  model.save('testingModel')
  convert_tf_to_tflite('/content/testingModel',[1,len(aug3[0]),len(aug3[0][0])], X_test,'whole_model',id_num,val_loss,val_accuracy)



# uncomment this to run training model as a whole(this means that this model can determine the motion as a whole but not specify what part if correct or not)
streamlined_process('/content/drive/MyDrive/Colab Notebooks/correct_new_2.txt','/content/drive/MyDrive/Colab Notebooks/wrong_new_2.txt')




NameError: name 'txt_pre_process' is not defined

In [2]:
# this is for the individual models
# this is experimental
def streamlined_process(correct_execution,noise_data,loop=5):
  id_num = str(rand.randint(1000,9999))
  base_data = txt_pre_process(correct_execution,1,False,4)
  base_data_noise = txt_pre_process(noise_data,0,False,4)

  temp_correct_start=[]
  temp_correct_wrong=[]

  data = base_data[0]
  data_noise = base_data_noise[0]



  print('')
  print('----------------------------correct data augmentation ----------------------------')
  aug = common_length_sequence(data)
  aug2 = apply_z_score(aug,1)
  aug3 = paddingV2(aug2)
  aug4 = populate_0_input(aug3,data_noise)
  aug4 = np.array(aug4)
  aug4 = aug4.reshape(-1,len(aug3[0]),len(aug3[0][0]))
  combined_inputs = np.concatenate((aug4,aug3), axis = 0)
  combined_inputs= aug4
  print('concat -> ', len(combined_inputs))



  print('')
  print('----------------------------data noise data augmentation ----------------------------')
  aug_noise_data1 = paddingV2(data_noise,len(aug3[0]))
  aug_noise_data2 = populate_0_input(aug_noise_data1,data_noise)
  aug_noise_data3 = aug_noise_data2[0:len(combined_inputs)]
  aug_noise_data4 = np.array(aug_noise_data3)
  aug_noise_data5 = aug_noise_data4.reshape(-1,len(aug_noise_data4[0]),len(aug_noise_data4[0][0]))

  aug_noise_data6 = data_aug_sensitivity(aug4,aug_noise_data5,1,1,int(len(combined_inputs)*.35))
  aug_noise_data5 = np.concatenate((aug_noise_data6,aug_noise_data5), axis = 0)
  # aug_noise_data6 = data_aug_coor_sensitivity(aug4)
  # aug_noise_data5 = np.concatenate((aug_noise_data6,aug_noise_data5), axis = 0)


  correct_data_set = []
  noise_data_set = []
  rand_batches = []

  print('testtesttesttest=======>',len(aug_noise_data5))


  for x in range(14):
    correct_data_set.append([])
    noise_data_set.append([])
    rand_batches.append([])


  for exercise in combined_inputs:

    left_upper_arm = []
    left_lower_arm = []
    left_hand = []
    right_upper_arm = []
    right_lower_arm = []
    right_hand = []
    left_upper_leg = []
    left_lower_leg = []
    left_feet = []
    right_upper_leg = []
    right_lower_leg = []
    right_feet = []
    head =[]
    body = []



#------------------------ generating the correct input of certain part------------------------------
    for sequence in exercise:
      # 11,13
      left_upper_arm.append([sequence[22],sequence[23],sequence[26],sequence[27]])
      # 13,15
      left_lower_arm.append([sequence[26],sequence[27],sequence[30],sequence[31]])
      # 15,17,19,21
      left_hand.append([sequence[30],sequence[31],sequence[34],sequence[35],sequence[38],sequence[39],sequence[42],sequence[43]])

      # 12,14
      right_upper_arm.append([sequence[24],sequence[25],sequence[28],sequence[29]])
      # 14,16
      right_lower_arm.append([sequence[28],sequence[29],sequence[32],sequence[33]])
      # 16,18,20,22
      right_hand.append([sequence[32],sequence[33],sequence[36],sequence[37],sequence[40],sequence[41],sequence[44],sequence[45]])

      # 23,25
      left_upper_leg.append([sequence[46],sequence[47],sequence[50],sequence[51]])
      # 25,27
      left_lower_leg.append([sequence[50],sequence[51],sequence[54],sequence[55]])
      # 27,29,31
      left_feet.append([sequence[54],sequence[55],sequence[58],sequence[59],sequence[62],sequence[63]])

      # 24,26
      right_upper_leg.append([sequence[48],sequence[49],sequence[52],sequence[53]])
      # 26,28
      right_lower_leg.append([sequence[52],sequence[53],sequence[56],sequence[57]])
      # 28,30,32
      right_feet.append([sequence[56],sequence[57],sequence[60],sequence[61],sequence[64],sequence[65]])

      # 11,12,23,24
      body.append([sequence[22],sequence[23],sequence[24],sequence[25],sequence[46],sequence[47],sequence[48],sequence[49]])
      # 7,8,9,10
      head.append([sequence[14],sequence[15],sequence[16],sequence[17],sequence[18],sequence[19],sequence[20],sequence[21]])


    correct_data_set[0].append(left_upper_arm)
    correct_data_set[1].append(left_lower_arm)
    correct_data_set[2].append(left_hand)

    correct_data_set[3].append(right_upper_arm)
    correct_data_set[4].append(right_lower_arm)
    correct_data_set[5].append(right_hand)

    correct_data_set[6].append(left_upper_leg)
    correct_data_set[7].append(left_lower_leg)
    correct_data_set[8].append(left_feet)

    correct_data_set[9].append(right_upper_leg)
    correct_data_set[10].append(right_lower_leg)
    correct_data_set[11].append(right_feet)

    correct_data_set[12].append(body)
    correct_data_set[13].append(head)




  for x in range(len(correct_data_set)):
    correct_data_set[x] = np.array(correct_data_set[x])

  # for x in range(len(noise_data_set)):
  #   noise_data_set[x] = np.array(noise_data_set[x])

  print('len of correct data set ---->',len(correct_data_set[0]))
  print('len of noise data set ---->',len(noise_data_set[0]))

  correct_data_set_label = np.ones(len(combined_inputs))
  noise_data_set_label = np.zeros(len(aug_noise_data5))

  # for x in range(len(correct_data_set)):
  #   rand_batches[x]=concatenate_randomize_batches(correct_data_set[x],correct_data_set_label,noise_data_set[x],noise_data_set_label)



  loop = int(len(aug_noise_data5)/len(combined_inputs))


  data_set_name=['left_upper_arm','left_lower_arm','left_hand','right_upper_arm','right_lower_arm','right_hand','left_upper_leg','left_lower_leg','left_feet','right_upper_leg','right_lower_leg','right_feet','head','body']
  for x in range(len(data_set_name)-1):
    print('progress -> ',x,'/',len(data_set_name)-1)



    best_val_loss = float('inf')
    best_val_accuracy = 0.0
    best_model = None

    for y in range(loop):

      noise_data_set=[]


    #------------------------------ generating noise ---------------------------------------
      for exercise in aug_noise_data5[len(correct_data_set[x])*y:len(correct_data_set[x]) + (len(correct_data_set[x])*y)]:
      # for exercise in aug_noise_data5[0:450]:
        # print('batchin noise ->',len(correct_data_set[x])*y,'<==--==>',len(correct_data_set[x]) + (len(correct_data_set[x])*y))


        left_upper_arm = []
        left_lower_arm = []
        left_hand = []
        right_upper_arm = []
        right_lower_arm = []
        right_hand = []
        left_upper_leg = []
        left_lower_leg = []
        left_feet = []
        right_upper_leg = []
        right_lower_leg = []
        right_feet = []
        head =[]
        body = []



        for sequence in exercise:
          # 11,13
          # print(sequence[22])
          if x == 0:
            left_upper_arm.append([sequence[22],sequence[23],sequence[26],sequence[27]])
          # 13,15
          if x == 1:
            left_lower_arm.append([sequence[26],sequence[27],sequence[30],sequence[31]])
          # 15,17,19,21
          if x == 2:
            left_hand.append([sequence[30],sequence[31],sequence[34],sequence[35],sequence[38],sequence[39],sequence[42],sequence[43]])

          # 12,14
          if x == 3:
            right_upper_arm.append([sequence[24],sequence[25],sequence[28],sequence[29]])
          # 14,16
          if x == 4:
            right_lower_arm.append([sequence[28],sequence[29],sequence[32],sequence[33]])
          # 16,18,20,22
          if x == 5:
            right_hand.append([sequence[32],sequence[33],sequence[36],sequence[37],sequence[40],sequence[41],sequence[44],sequence[45]])

          # 23,25
          if x == 6:
            left_upper_leg.append([sequence[46],sequence[47],sequence[50],sequence[51]])
          # 25,27
          if x == 7:
            left_lower_leg.append([sequence[50],sequence[51],sequence[54],sequence[55]])
          # 27,29,31
          if x == 8:
            left_feet.append([sequence[54],sequence[55],sequence[58],sequence[59],sequence[62],sequence[63]])

          # 24,26
          if x == 9:
            right_upper_leg.append([sequence[48],sequence[49],sequence[52],sequence[53]])
          # 26,28
          if x == 10:
            right_lower_leg.append([sequence[52],sequence[53],sequence[56],sequence[57]])
          # 28,30,32
          if x == 11:
            right_feet.append([sequence[56],sequence[57],sequence[60],sequence[61],sequence[64],sequence[65]])

          # 11,12,23,24
          if x == 12:
            body.append([sequence[22],sequence[23],sequence[24],sequence[25],sequence[46],sequence[47],sequence[48],sequence[49]])
          # 7,8,9,10
          if x == 13:
            head.append([sequence[14],sequence[15],sequence[16],sequence[17],sequence[18],sequence[19],sequence[20],sequence[21]])

        if x == 0:
          noise_data_set.append(left_upper_arm)
        if x == 1:
          noise_data_set.append(left_lower_arm)
        if x == 2:
          noise_data_set.append(left_hand)

        if x == 3:
          noise_data_set.append(right_upper_arm)
        if x == 4:
          noise_data_set.append(right_lower_arm)
        if x == 5:
          noise_data_set.append(right_hand)

        if x == 6:
          noise_data_set.append(left_upper_leg)
        if x == 7:
          noise_data_set.append(left_lower_leg)
        if x == 8:
          noise_data_set.append(left_feet)


        if x == 9:
          noise_data_set.append(right_upper_leg)
        if x == 10:
          noise_data_set.append(right_lower_leg)
        if x == 11:
          noise_data_set.append(right_feet)


        if x == 12:
          noise_data_set.append(body)
        if x == 13:
          noise_data_set.append(head)

# ===================================================================================================================================================

      print('correct->',len(correct_data_set[x]),'  incorrect->',len(noise_data_set[x]))

      rand_batches=concatenate_randomize_batches(correct_data_set[x],correct_data_set_label,noise_data_set,noise_data_set_label)





      print('loop -> ',y,'/',loop)
      X_train, X_test, y_train, y_test = train_test_split(rand_batches[0], rand_batches[1], test_size=0.2, random_state=42)

      # history = model.fit(X_train[y*len(rand_batches[x][0]):len(rand_batches[x][0])*(y+1)], y_train[y*len(rand_batches[x][1]):len(rand_batches[x][1])*(y+1)], epochs=200, batch_size =128 , validation_data=(X_test[y*len(rand_batches[x][0]):len(rand_batches[x][0])*(y+1)], y_test[y*len(rand_batches[x][1]):len(rand_batches[x][1])*(y+1)]), callbacks=[custom_early_stopping])

      if y == 0:
        model_base_modifier = 10
        model = Sequential()
        model.add(LSTM(model_base_modifier+len(aug4[0]), return_sequences=True, activation='relu',  input_shape=(len(correct_data_set[x][0]), len(correct_data_set[x][0][0]))))
        model.add(LSTM(model_base_modifier+len(aug4[0]) + int(len(aug4[0]) - int(len(aug4[0]) * .4)), return_sequences=True,  activation='relu' ))
        model.add(Bidirectional(LSTM(model_base_modifier+len(aug4[0]) - int(len(aug4[0]) - int(len(aug4[0]) * .5)), return_sequences=True, dropout=0.4, recurrent_dropout=0.4, activation='relu')))
        model.add(LSTM(model_base_modifier+len(aug4[0]) - int(len(aug4[0]) - int(len(aug4[0]) * .4)), return_sequences=False,  activation='relu'))
        # model.add(BatchNormalization())
        model.add(Dense(model_base_modifier+len(aug4[0]) - int(len(aug4[0]) - int(len(aug4[0]) * .4)), activation='relu'))

        model.add(Dense(1,activation='sigmoid'))

        custom_early_stopping = CustomEarlyStopping(accuracy_threshold=0.97, loss_threshold=0.05)
        model.compile(optimizer = 'Adam' , loss = 'binary_crossentropy', metrics=['accuracy'])



      history = model.fit(X_train, y_train, epochs=150, batch_size =128 , validation_data=(X_test, y_test), callbacks=[custom_early_stopping])

      val_loss = min(history.history['val_loss'])
      val_accuracy = max(history.history['val_accuracy'])

      if val_loss < best_val_loss and val_accuracy > best_val_accuracy:
        best_model = model.get_weights()
        best_val_loss = val_loss
        best_val_accuracy = val_accuracy

      model.set_weights(best_model)



    model.save('testingModel')


    # X_train = X_train[x].astype(np.float32)
    # X_test = X_test[x].astype(np.float32)

    # convert_tf_to_tflite('/content/testingModel',[1,len(X_train[0]),len(X_train[0][0])], X_test,data_set_name[x],id_num,val_loss,val_accuracy)
    # convert_tf_to_tflite('/content/testingModel',[1,len(correct_data_set[x][0]),len(correct_data_set[x][0][0])], X_test,data_set_name[x],id_num,val_loss,val_accuracy)


    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)
    model.save('testingModel')

    print('param 1 ->',len(noise_data_set[0]) )
    print('param 1 ->',len(noise_data_set[0][0]))
    print('current x ->',x)
    convert_tf_to_tflite('/content/testingModel',[1,len(noise_data_set[0]),len(noise_data_set[0][0])], X_test,data_set_name[x],id_num,best_val_loss,best_val_accuracy)




    clear_output(wait=True)





# uncomment this to run the individual landmarks training training(this means that it can determine individual parts of the body if spefic parts are incorrect or correct)
# streamlined_process('/content/drive/MyDrive/Colab Notebooks/correct_new_2.txt','/content/drive/MyDrive/Colab Notebooks/wrong_new_2.txt')
3

3

In [80]:
import tensorflow as tf
import cv2
import numpy as np

# Mock skeletal coordinates (replace this with your actual data)
base_data = txt_pre_process('firstExerciseFitguide(correct).txt',1,False,4)



# skeletal_coordinates = np.array([[50, 50], [50, 100], [100, 100], [100, 50]])
skeletal_coordinates = base_data[0][0][0]
tempCtr = 0
temp1=[]

for x in range(33):
    temp1.append([skeletal_coordinates[tempCtr],skeletal_coordinates[tempCtr+1]])
    tempCtr = tempCtr + 2


image_size = (200, 200, 3)
black_background = np.zeros(image_size, dtype=np.uint8)

for i in range(len(temp1) - 1):
    # Convert coordinates to integers only when drawing
    start_point = tuple(map(int, skeletal_coordinates[i]))
    end_point = tuple(map(int, skeletal_coordinates[i + 1]))

    cv2.line(black_background, start_point, end_point, (255, 255, 255), 2)

# Create a black background image


# Draw lines on the black background based on skeletal coordinates
# for i in range(len(skeletal_coordinates) - 1):
#     cv2.line(black_background, tuple(skeletal_coordinates[i]), tuple(skeletal_coordinates[i + 1]), (255, 255, 255), 2)

# Display the image
cv2.imshow('Skeletal Image', black_background)
cv2.waitKey(0)
cv2.destroyAllWindows()


TypeError: 'float' object is not iterable

In [83]:
import cv2
import numpy as np
height = 500
width = 200
image_size = (height, width, 3)

highestSeq = 0
temp_H_seq = []

black_background = np.zeros(image_size, dtype=np.uint8)


base_data = txt_pre_process('firstExerciseFitguide(correct).txt', 1, False, 4)
base_data = base_data[0][40]
temp1=[]
tempCtr = 0
# print("len-->",len(base_data))
# print(len(base_data[0]))

for sequenceList in base_data:
    
    print(sequenceList,"----",highestSeq)
    if len(sequenceList) >= highestSeq:
        highestSeq = sequenceList

for sequenceList in base_data[0]:
    if len(sequenceList) == highestSeq:
        temp_H_seq.append(sequenceList)
    




for y in range(len(sequenceList)):
    black_background = np.zeros(image_size, dtype=np.uint8)

    tempCtr =0
    temp1 = []
    for x in range(33):
        temp1.append([int(temp_H_seq[y][tempCtr]*width),int(temp_H_seq[y][tempCtr+1]*height)])
        tempCtr = tempCtr + 2
        print(len(temp1))

    # left foot
    cv2.line(black_background, temp1[32], temp1[30], (255, 255, 255), 2)
    cv2.line(black_background, temp1[30], temp1[28], (255, 255, 255), 2)
    cv2.line(black_background, temp1[32], temp1[28], (255, 255, 255), 2)

    # right Foot
    cv2.line(black_background, temp1[27], temp1[31], (255, 255, 255), 2)
    cv2.line(black_background, temp1[31], temp1[29], (255, 255, 255), 2)
    cv2.line(black_background, temp1[29], temp1[27], (255, 255, 255), 2)

    # left leg
    cv2.line(black_background, temp1[28], temp1[26], (255, 255, 255), 2)
    cv2.line(black_background, temp1[26], temp1[24], (255, 255, 255), 2)

    # right leg
    cv2.line(black_background, temp1[27], temp1[25], (255, 255, 255), 2)
    cv2.line(black_background, temp1[25], temp1[23], (255, 255, 255), 2)

    # body
    cv2.line(black_background, temp1[24], temp1[23], (255, 255, 255), 2)
    cv2.line(black_background, temp1[23], temp1[11], (255, 255, 255), 2)
    cv2.line(black_background, temp1[11], temp1[12], (255, 255, 255), 2)
    cv2.line(black_background, temp1[12], temp1[24], (255, 255, 255), 2)

    # left arm
    cv2.line(black_background, temp1[12], temp1[14], (255, 255, 255), 2)
    cv2.line(black_background, temp1[14], temp1[16], (255, 255, 255), 2)

    # left hand
    cv2.line(black_background, temp1[16], temp1[18], (255, 255, 255), 2)
    cv2.line(black_background, temp1[18], temp1[20], (255, 255, 255), 2)
    cv2.line(black_background, temp1[20], temp1[16], (255, 255, 255), 2)
    cv2.line(black_background, temp1[16], temp1[22], (255, 255, 255), 2)

    # right arm
    cv2.line(black_background, temp1[11], temp1[13], (255, 255, 255), 2)
    cv2.line(black_background, temp1[13], temp1[15], (255, 255, 255), 2)

    # right hand
    cv2.line(black_background, temp1[15], temp1[17], (255, 255, 255), 2)
    cv2.line(black_background, temp1[17], temp1[19], (255, 255, 255), 2)
    cv2.line(black_background, temp1[19], temp1[15], (255, 255, 255), 2)
    cv2.line(black_background, temp1[15], temp1[21], (255, 255, 255), 2)

    # face
    cv2.line(black_background, temp1[7], temp1[3], (255, 255, 255), 2)
    cv2.line(black_background, temp1[3], temp1[2], (255, 255, 255), 2)
    cv2.line(black_background, temp1[2], temp1[1], (255, 255, 255), 2)
    cv2.line(black_background, temp1[0], temp1[4], (255, 255, 255), 2)
    cv2.line(black_background, temp1[0], temp1[4], (255, 255, 255), 2)
    cv2.line(black_background, temp1[4], temp1[5], (255, 255, 255), 2)
    cv2.line(black_background, temp1[5], temp1[6], (255, 255, 255), 2)
    cv2.line(black_background, temp1[6], temp1[7], (255, 255, 255), 2)
    cv2.line(black_background, temp1[7], temp1[8], (255, 255, 255), 2)

    # mouth
    cv2.line(black_background, temp1[10], temp1[9], (255, 255, 255), 2)




    cv2.imshow('Skeletal Image', black_background)
    cv2.waitKey(0)
    cv2.destroyAllWindows()








[0.59652624, 0.02667674, 0.58421618, 0.00168468, 0.59267123, 0.00101575, 0.59962849, 0.0, 0.54127683, 0.00120362, 0.51851573, 0.00065259, 0.49172178, 0.00029337, 0.57099637, 0.00786889, 0.41215745, 0.01199619, 0.59732561, 0.04739432, 0.54323311, 0.0482398, 0.73056635, 0.13932864, 0.19639467, 0.14795567, 0.86508043, 0.31035553, 0.08993779, 0.32513696, 0.9544216, 0.47049416, 0.03687205, 0.48484136, 1.0, 0.52071026, 0.0, 0.53177139, 0.95038218, 0.52437813, 0.04261503, 0.53557986, 0.91838214, 0.50537482, 0.07422279, 0.51945688, 0.66757966, 0.4674395, 0.34961041, 0.47650171, 0.67116715, 0.71119287, 0.38583393, 0.70923163, 0.69751153, 0.91788413, 0.45857493, 0.9158063, 0.67752156, 0.94017888, 0.49370885, 0.93859664, 0.75245725, 0.99957656, 0.3469685, 1.0] ---- 0
[0.60607397, 0.02864242, 0.60260547, 0.00267221, 0.61245055, 0.00228204, 0.62116926, 0.00160955, 0.55862231, 0.00137912, 0.5361607, 0.0005643, 0.50949435, 0.0, 0.5979642, 0.00893795, 0.42914406, 0.00988336, 0.60940093, 0.04877043, 0.

TypeError: '>=' not supported between instances of 'int' and 'list'

In [78]:
# import cv2
# import numpy as np
# height = 500
# width = 200
# image_size = (height, width, 3)

# black_background = np.zeros(image_size, dtype=np.uint8)


# base_data = txt_pre_process('firstExerciseFitguide(correct).txt', 1, False, 4)
# base_data = base_data[0]
# temp1=[]
# tempCtr = 0

# print(len(base_data))


# for y in range(len(base_data)):
#     for x in range(33):
#         temp1.append([int(base_data[y][tempCtr]*width),int(base_data[y][tempCtr+1]*height)])
#         tempCtr = tempCtr + 2
#         print(temp1)

#     # left foot
#     cv2.line(black_background, temp1[32], temp1[30], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[30], temp1[28], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[32], temp1[28], (255, 255, 255), 2)

#     # right Foot
#     cv2.line(black_background, temp1[27], temp1[31], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[31], temp1[29], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[29], temp1[27], (255, 255, 255), 2)

#     # left leg
#     cv2.line(black_background, temp1[28], temp1[26], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[26], temp1[24], (255, 255, 255), 2)

#     # right leg
#     cv2.line(black_background, temp1[27], temp1[25], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[25], temp1[23], (255, 255, 255), 2)

#     # body
#     cv2.line(black_background, temp1[24], temp1[23], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[23], temp1[11], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[11], temp1[12], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[12], temp1[24], (255, 255, 255), 2)

#     # left arm
#     cv2.line(black_background, temp1[12], temp1[14], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[14], temp1[16], (255, 255, 255), 2)

#     # left hand
#     cv2.line(black_background, temp1[17], temp1[18], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[18], temp1[20], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[20], temp1[16], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[16], temp1[23], (255, 255, 255), 2)

#     # right arm
#     cv2.line(black_background, temp1[11], temp1[13], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[13], temp1[15], (255, 255, 255), 2)

#     # right hand
#     cv2.line(black_background, temp1[15], temp1[17], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[17], temp1[19], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[19], temp1[15], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[15], temp1[21], (255, 255, 255), 2)

#     # face
#     cv2.line(black_background, temp1[7], temp1[3], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[3], temp1[2], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[2], temp1[1], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[0], temp1[4], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[0], temp1[4], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[4], temp1[5], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[5], temp1[6], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[6], temp1[7], (255, 255, 255), 2)
#     cv2.line(black_background, temp1[7], temp1[8], (255, 255, 255), 2)

#     # mouth
#     cv2.line(black_background, temp1[10], temp1[9], (255, 255, 255), 2)












#     cv2.imshow('Skeletal Image', black_background)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()








120


TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [5]:

# Assuming the coordinates range from 0 to 1
skeletal_coordinates = base_data[0][0][0]


# Scale relative coordinates to pixel coordinates
scaled_coordinates = np.array(skeletal_coordinates) * np.array(image_size[:2])

tempCtr = 0
temp1=[]

for x in range(33):
    temp1.append([scaled_coordinates[tempCtr],scaled_coordinates[tempCtr+1]])
    tempCtr = tempCtr + 2

# Convert to integers for drawing
scaled_coordinates = scaled_coordinates.astype(int)

# Create a black background image
image_size = (200, 200, 3)
black_background = np.zeros(image_size, dtype=np.uint8)

# Draw lines on the black background based on scaled coordinates
for i in range(len(scaled_coordinates) - 1):
    start_point = tuple(scaled_coordinates[i])
    end_point = tuple(scaled_coordinates[i + 1])

    cv2.line(black_background, start_point, end_point, (255, 255, 255), 2)

# Display the image
cv2.imshow('Skeletal Image', black_background)
cv2.waitKey(0)
cv2.destroyAllWindows()


NameError: name 'base_data' is not defined